In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
plt.style.use('ggplot')
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import scipy as sp
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/kaggle/input/zomato-bangalore-restaurants/zomato.csv")

In [ ]:
# Output head
df.head()

In [ ]:
#output info
df.info()

**Columns description**

url: contains the url of the restaurant in the zomato website

address: contains the address of the restaurant in Bengaluru

name: contains the name of the restaurant

online_order: whether online ordering is available in the restaurant or not

book_table: table book option available or not

rate: contains the overall rating of the restaurant out of 5

votes: contains total number of rating for the restaurant as of the above mentioned date

phone: contains the phone number of the restaurant

location: contains the neighborhood in which the restaurant is located

rest_type: restaurant type

dish_liked: dishes people liked in the restaurant

cuisines: food styles, separated by comma

approx_cost(for two people): contains the approximate cost for meal for two people

reviews_list: list of tuples containing reviews for the restaurant, each tuple

menu_item: contains list of menus available in the restaurant

listed_in(type): type of meal

listed_in(city): contains the neighborhood in which the restaurant is listed

In [ ]:
#Deleting Unnnecessary Columns
zomato=df.drop(['url','dish_liked','phone'],axis=1) 

In [ ]:
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

In [ ]:
#Remove the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)

In [ ]:
zomato.info()
zomato.head()

找出出现频率最高的商店（最受欢迎的商店）排名前50
可以看出排名最高的是Cafe Coffee Day，其次是Onesta

In [ ]:
# 建立图片
plt.figure(figsize=(15,10))
# 获取各个商店的名称,并在数据中进行统计
chains=zomato['name'].value_counts()[:50]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most famous restaurants TOP 50")
plt.xlabel("Number of outlets")
plt.show()

统计餐厅的类型
可以看出餐厅类型中Quick Bites是最多的，而Casual Dinning其次

In [ ]:
plt.figure(figsize=(7,7))
rest=zomato['rest_type'].value_counts()[:20]
sns.barplot(rest,rest.index)
plt.title("Restaurant types")
plt.xlabel("count")
plt.show()

统计在餐厅中两个人消费额度的一个分布情况
可以看出两个人的消费额度主要分布在200-800的区间

In [ ]:
cost_dist=zomato[['rate','approx_cost(for two people)','online_order']].dropna()
cost_dist['rate']=cost_dist['rate'].apply(lambda x: float(x.split('/')[0]) if len(x)>3 else 0)
cost_dist['approx_cost(for two people)']=cost_dist['approx_cost(for two people)'].apply(lambda x: int(x.replace(',','')))
plt.figure(figsize=(8,6))
sns.distplot(cost_dist['approx_cost(for two people)'])
plt.show()

统计餐厅接受在线点餐的分布
可以看到有大约64.8%的餐厅接受在线点餐，而35.2%的不接受在线点餐

In [ ]:
x = zomato['online_order'].value_counts()
colors = ['orange', 'brown']
trace=go.Pie(labels=x.index,values=x,
            marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))
layout=go.Layout(title="Online orders",width=500,height=500)
fig=go.Figure(data=[trace],layout=layout)
py.iplot(fig, filename='pie_chart_subplots')

统计餐厅是否接受table booking
可以看出只有14.8%的餐厅接受订餐服务，而85.2%的餐厅不接受订餐服务

In [ ]:
x = zomato['book_table'].value_counts()
colors = ['#FEBFB3', '#E1396C']
trace=go.Pie(labels=x.index,values=x,
            marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))
layout=go.Layout(title="Table booking",width=500,height=500)
fig=go.Figure(data=[trace],layout=layout)
py.iplot(fig, filename='pie_chart_subplots')


统计客户给餐厅评价分数的分布
可以观察到主要分布在3.5-4之间

In [ ]:
plt.figure(figsize=(8,6))
rating=zomato['rate'].dropna().apply(lambda x : float(x.split('/')[0]) if (len(x)>3)  else np.nan ).dropna()
sns.distplot(rating,bins=20)
plt.show()

统计最受欢迎的菜肴
可以看出最受欢迎的来自于北印度的菜肴

In [ ]:
plt.figure(figsize=(7,7))
cuisines=zomato['cuisines'].value_counts()[:10]
sns.barplot(cuisines,cuisines.index)
plt.xlabel('Count')
plt.title("Most popular cuisines of Bangalore")
plt.show()

根据餐厅种类来画出每个种类餐厅的菜肴词云

只保留有dish_like词条的有效数据

In [ ]:
#Deleting Unnnecessary Columns
zomato1 = df.drop(['url','phone'],axis=1)
#Removing the Duplicates
zomato1.duplicated().sum()
zomato1.drop_duplicates(inplace=True)
#Remove the NaN values from the dataset
zomato1.isnull().sum()
zomato1.dropna(how='any',inplace=True)
zomato1.info()

In [ ]:
zomato1['dish_liked']=zomato1['dish_liked'].apply(lambda x : x.split(',') if type(x)==str else [''])
#x=df.groupby('rest_type',as_index=False)['dish_liked'].agg('sum')
#x['dish_liked']=x['dish_liked'].apply(lambda x : list(filter(lambda a : a!='',x)))
rest=zomato1['rest_type'].value_counts()[:9].index
def produce_wordcloud(rest):
    
    plt.figure(figsize=(20,30))
    for i,r in enumerate(rest):
        plt.subplot(3,3,i+1)
        corpus=zomato1[zomato1['rest_type']==r]['dish_liked'].values.tolist()
        corpus=','.join(x  for list_words in corpus for x in list_words)
        wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=1500, height=1500).generate(corpus)
        plt.imshow(wordcloud)
        plt.title(r)
        plt.axis("off")
        
print(rest)
        
        
produce_wordcloud(rest)

对各个因素进行相关性的分析，输出相关性矩阵的热力图
相关性越大，说明这两个因素越线性相关 正的表示正相关，负的表示负相关 可以看出rate和vote、cuisines以及reviews_list相关性较高



In [ ]:
zomato2 = zomato
#Changing the Columns Names
zomato2.columns
zomato2 = zomato2.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})
#Some Transformations
zomato2['cost'] = zomato2['cost'].astype(str)
zomato2['cost'] = zomato2['cost'].apply(lambda x: x.replace(',','.'))
zomato2['cost'] = zomato2['cost'].astype(float)
#Removing '/5' from Rates
zomato2['rate'].unique()
zomato2 = zomato2.loc[zomato2.rate !='NEW']
zomato2 = zomato2.loc[zomato2.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato2.rate = zomato2.rate.apply(remove_slash).str.strip().astype('float')
# Adjust the column names
zomato2.name = zomato2.name.apply(lambda x:x.title())
zomato2.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato2.book_table.replace(('Yes','No'),(True, False),inplace=True)
#Encode the input Variables
def Encode(zomato2):
    for column in zomato2.columns[~zomato2.columns.isin(['rate', 'cost', 'votes'])]:
        zomato2[column] = zomato2[column].factorize()[0]
    return zomato2

zomato2_en = Encode(zomato2.copy())
#Get Correlation between different variables
corr = zomato2_en.corr(method='kendall')
plt.figure(figsize=(15,8))
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
zomato2.info()

In [ ]:
#Online order vs Rate
plt.rcParams['figure.figsize'] = (13, 9)
Y = pd.crosstab(zomato2['rate'], zomato2['online_order'])
Y.div(Y.sum(1).astype(float), axis = 0).plot(kind = 'bar', stacked = True,color=['orange','brown'])
plt.title('Online order vs Rate', fontweight = 30, fontsize = 20)
plt.legend(loc="upper right")
plt.show()

In [ ]:
#Table booking Rate vs Rate
plt.rcParams['figure.figsize'] = (13, 9)
Y = pd.crosstab(zomato2['rate'], zomato2['book_table'])
Y.div(Y.sum(1).astype(float), axis = 0).plot(kind = 'bar', stacked = True,color=['orange','brown'])
plt.title('Table booking vs Rate', fontweight = 30, fontsize = 20)
plt.legend(loc="upper right")
plt.show()

Location

In [ ]:
sns.countplot(zomato2['city'])
sns.countplot(zomato2['city']).set_xticklabels(sns.countplot(zomato2['city']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(15,8)
plt.title('Location wise count for restaurants')

Location VS rate

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
loc_plt=pd.crosstab(zomato2['rate'],zomato2['city'])
loc_plt.plot(kind='bar',stacked=True);
plt.title('Location VS Rate',fontsize=15,fontweight='bold')
plt.ylabel('Number',fontsize=10,fontweight='bold')
plt.xlabel('Rate',fontsize=10,fontweight='bold')
plt.xticks(fontsize=10,fontweight='bold')
plt.yticks(fontsize=10,fontweight='bold');
plt.legend();

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
loc_plt=pd.crosstab(zomato2['rate'],zomato2['type'])
loc_plt.plot(kind='bar',stacked=True);
plt.title('Type VS Rate',fontsize=15,fontweight='bold')
plt.ylabel('Number',fontsize=10,fontweight='bold')
plt.xlabel('Rate',fontsize=10,fontweight='bold')
plt.xticks(fontsize=10,fontweight='bold')
plt.yticks(fontsize=10,fontweight='bold');
plt.legend();

Rate VS votes

In [ ]:
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(1,1,1)
sns.scatterplot(x=zomato2['rate'],y=zomato2['votes'],ax=ax1)
slope,inter,r_value,p_value,std_err = sp.stats.linregress(x=zomato2['rate'],y=zomato2['votes'])
x = np.array(zomato2['rate'])
y = np.array(zomato2['rate'].apply(lambda y:y*slope + inter))
ax1.plot(x,y)
plt.legend(labels=['slope : {}\nIntercept : {}\nR-Value : {}\nP-Value : {}\nstd-err : {}'.format(slope,inter,r_value,p_value,std_err),],loc=2)
print('There is a strong Relation between rate and votes, higher rate will attract more people to give votes')